In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os

In [7]:
device =  "cuda" if torch.cuda.is_available() else "cpu" 
device

'cuda'

In [19]:
block_size = 128
batch_size = 16
learning_rate = 1e-5
max_iter = 30000
eval_iters = 250
n_embed = 384
n_head = 6
dropout = 0.2
n_layer = 6

In [20]:
with open("/kaggle/input/wizard-of-oz-txt/wizard_of_oz.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [21]:
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
74


In [22]:
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

In [23]:
encode = lambda s: [string_to_int[ch] for ch in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

In [24]:
data = torch.tensor(encode(text), dtype=torch.long)

In [25]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [26]:
def get_batch(split):
  data = train_data if split == "train" else val_data
  idx  = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in idx])
  y = torch.stack([data[i+1:i+block_size+1] for i in idx])
  x, y = x.to(device), y.to(device)
  return x, y

In [27]:
x, y = get_batch("train")

In [28]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ["train", "val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [34]:
class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.values = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        v = self.values(x)
        out = wei @ v
        return out
        
f
class MultiHeadAttention(nn.Module):
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.linear = nn.Linear(head_size * num_heads, n_embed)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.linear(out))
        return out

class FeedForward(nn.Module):
    
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
        
    def forward(self,x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])


        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
        self.apply(self.__init_weights)


    def __init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, index, targets=None):
        B, T = index.shape

        tok_embed = self.token_embedding_table(index)
        pos_embed = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_embed + pos_embed
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)



        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

In [ ]:
model = GPTLanguageModel(vocab_size=len(chars))
model = model.to(device)

In [ ]:
context = torch.zeros((batch_size, block_size), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, max_new_tokens=20)[0].tolist())
print(generated_chars)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(1, max_iter+1):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step : {iter}\tTrain loss : {losses['train']:.3f},\tVal loss : {losses['val']:.3f}")
    
    xb, yb = get_batch("train")
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())
torch.save(model.state_dict(), "./")

In [ ]:
def user_prompt(model, context=None, max_new_tokens=1000):

    if context is None:
        user_input = input("Enter your prompt: ")
        context = torch.tensor(encode(user_input), dtype=torch.long, device=device)

    # Generate new tokens
    generated_index = decode(model.generate(context, max_new_tokens=max_new_tokens)[0].tolist())

    # Print the generated text
    print(generated_text)

    #return generated_text

In [ ]:
user_prompt(model=model, "hello")